<a href="https://colab.research.google.com/github/navruzbek1/Datasets/blob/main/california_house.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import sklearn

In [ ]:
df=pd.read_csv("/content/loyiha1.csv",index_col=0)

In [ ]:
from sklearn.model_selection import train_test_split
train_set,test_set =train_test_split(df,train_size=0.8,random_state=21)

In [ ]:
housing=train_set.drop("median_house_value",axis=1)
housing_label=train_set["median_house_value"]
housing_num=housing.drop("ocean_proximity",axis=1)

ishlatmadim


In [ ]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy="median")
X=imputer.fit_transform(housing.drop("ocean_proximity",axis=1))

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
# bizga kerak ustunlar indekslari
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class assalom(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self # bizni funksiyamiz faqat transformer. estimator emas
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room: # add_bedrooms_per_room ustuni ixtiyoriy bo'ladi
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler

num_pipeline=Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("add_trans",assalom()),
    ("st_scale",StandardScaler())]
)#sonli hisoblashlar uchun

num_pipeline.fit_transform(housing_num)

In [ ]:
from sklearn.compose import ColumnTransformer

sonli_qator=list(housing_num)
matnli_qator=["ocean_proximity"]

full_pipeline=ColumnTransformer([
       ("num",num_pipeline,sonli_qator),
       ("matn",OneHotEncoder(),matnli_qator) 
])#matnli hisoblashlar uchun

In [ ]:
X_prepared=full_pipeline.fit_transform(housing)

In [ ]:
from sklearn.linear_model import LinearRegression
LR_model=LinearRegression()

In [ ]:
y=train_set["median_house_value"]

In [ ]:
LR_model.fit(X_prepared,y)

LinearRegression()

In [ ]:
X_prepared.shape

(16512, 16)

In [ ]:
test_data=train_set.sample(10)
test_data

In [ ]:
test_label=y.loc[test_data.index]
test_label

In [ ]:
test_data_predicted=full_pipeline.transform(test_data)
test_data_predicted[0,:]

In [ ]:
predicted_labels=LR_model.predict(test_data_predicted)
predicted_labels